In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

### data load

In [12]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("")
print("train.num =", mnist.train.num_examples,
     ", test.num =", mnist.test.num_examples,
     ", validation.num =", mnist.validation.num_examples)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

train.num = 55000 , test.num = 10000 , validation.num = 5000


### hyper parameter 

In [13]:
# Hyper-Parameter
learning_rate = 1e-3
epochs = 30
batch_size = 100

In [14]:
# 입력과 정답을 위한 플레이스홀더 정의
X = tf.placeholder(tf.float32, [None, 784])  
T = tf.placeholder(tf.float32, [None, 10])  

# 입력층의 출력 값. 컨볼루션 연산을 위해 reshape 시킴
A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])  # image 28 x 28 x 1 (black / white)

### 컨볼루션층 1

In [15]:
# 1번째 컨볼루션 층, 3x3x32 필터
W2 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
b2 = tf.Variable(tf.random_normal([32]))

# 1번째 컨볼루션 연산을 통해 28 x 28 x 1 => 28 x 28 x 32
C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28 x 28 x 32 => 14 x 14 x 32
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 완전연결층 

In [16]:
# 14 x 14 크기를 가진 32개의 activation map을 flatten 시킴
A2_flat = P2_flat = tf.reshape(A2, [-1, 14*14*32])

### 출력층

In [17]:
# 출력층
W3 = tf.Variable(tf.random_normal([14*14*32, 10], stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀 값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2_flat, W3) + b3

y = A3 = tf.nn.softmax(Z3)

In [18]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = Z3, labels = T))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [19]:
# batch_size x 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal(tf.argmax(A3, 1), tf.argmax(T, 1))

# batch_size x 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

In [20]:
with tf.Session() as sess:  # with를 쓰면 close 하지 않아도 됨
    sess.run(tf.global_variables_initializer())  # 변수 노드 (tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):  # 30번 반복 수행
        total_batch = int(mnist.train.num_examples / batch_size)  # 55000 / 100
        
        for step in range(total_batch):
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now()
    
    print("")
    print("Elapsed Time => ", end_time-start_time)
    
    # Accuracy 확인
    test_x_data = mnist.test.images  # 10000 x 784
    test_t_data = mnist.test.labels  # 10000 x 10
    
    accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy =", accuracy_val)

epochs =  0 , step =  0 , loss_val =  2.9792135
epochs =  0 , step =  100 , loss_val =  1.4580147
epochs =  0 , step =  200 , loss_val =  0.5439368
epochs =  0 , step =  300 , loss_val =  0.33295545
epochs =  0 , step =  400 , loss_val =  0.41510475
epochs =  0 , step =  500 , loss_val =  0.44298196
epochs =  1 , step =  0 , loss_val =  0.3718492
epochs =  1 , step =  100 , loss_val =  0.39696172
epochs =  1 , step =  200 , loss_val =  0.4033933
epochs =  1 , step =  300 , loss_val =  0.27315497
epochs =  1 , step =  400 , loss_val =  0.32061127
epochs =  1 , step =  500 , loss_val =  0.11482627
epochs =  2 , step =  0 , loss_val =  0.22714698
epochs =  2 , step =  100 , loss_val =  0.30556133
epochs =  2 , step =  200 , loss_val =  0.14649706
epochs =  2 , step =  300 , loss_val =  0.2502213
epochs =  2 , step =  400 , loss_val =  0.33676213
epochs =  2 , step =  500 , loss_val =  0.23091894
epochs =  3 , step =  0 , loss_val =  0.2694272
epochs =  3 , step =  100 , loss_val =  0.2337

epochs =  26 , step =  300 , loss_val =  0.08342453
epochs =  26 , step =  400 , loss_val =  0.03633991
epochs =  26 , step =  500 , loss_val =  0.016219763
epochs =  27 , step =  0 , loss_val =  0.024167912
epochs =  27 , step =  100 , loss_val =  0.008900572
epochs =  27 , step =  200 , loss_val =  0.027936738
epochs =  27 , step =  300 , loss_val =  0.05866648
epochs =  27 , step =  400 , loss_val =  0.046255182
epochs =  27 , step =  500 , loss_val =  0.0578973
epochs =  28 , step =  0 , loss_val =  0.021993086
epochs =  28 , step =  100 , loss_val =  0.011280827
epochs =  28 , step =  200 , loss_val =  0.10884972
epochs =  28 , step =  300 , loss_val =  0.05047854
epochs =  28 , step =  400 , loss_val =  0.0047609187
epochs =  28 , step =  500 , loss_val =  0.020639656
epochs =  29 , step =  0 , loss_val =  0.022622809
epochs =  29 , step =  100 , loss_val =  0.017067138
epochs =  29 , step =  200 , loss_val =  0.0762514
epochs =  29 , step =  300 , loss_val =  0.026538825
epochs 